In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Function

In [34]:
scores = np.array([0.1, 0.3, 0.1, 0.2], dtype=np.float).reshape((4, 1))
labels = np.array([0, 1, 2, 0], dtype=np.float).reshape((4, 1))

In [35]:
A = np.tile(scores, scores.shape[0])
scores_diff = A - A.T

B = np.tile(labels, labels.shape[0])
labels_diff = B - B.T
labels_diff = (labels_diff > 0).astype(np.float)
np.fill_diagonal(labels_diff, 0.0)

sigmoid_like = -1.0 / (1.0 + np.exp(scores_diff))
print(sigmoid_like)
print(sigmoid_like * labels_diff)

[[-0.5        -0.549834   -0.5        -0.52497919]
 [-0.450166   -0.5        -0.450166   -0.47502081]
 [-0.5        -0.549834   -0.5        -0.52497919]
 [-0.47502081 -0.52497919 -0.47502081 -0.5       ]]
[[-0.         -0.         -0.         -0.        ]
 [-0.450166   -0.         -0.         -0.47502081]
 [-0.5        -0.549834   -0.         -0.52497919]
 [-0.         -0.         -0.         -0.        ]]


In [36]:
sorted_labels = labels.copy()
sorted_labels[::-1].sort(axis=0)
max_dcg = np.sum((np.power(2, sorted_labels) - 1) / np.log(np.arange(sorted_labels.shape[0]) + 2).reshape((sorted_labels.shape[0], 1)))
print(max_dcg)

5.23832434929


In [47]:
# compute gain by puting label i at position j
N = labels.shape[0]
DCG = np.tile(np.power(2, labels) - 1, N) / np.tile(np.log(np.arange(N) + 2), (N, 1))
print(DCG)

DCG_T = DCG.T
print(DCG_T)

delta_dcg = DCG + DCG_T - np.tile(DCG.diagonal().reshape((N, 1)), N) - np.tile(DCG.diagonal(), (N, 1))
delta_dcg = np.abs(delta_dcg)
print(delta_dcg)

[[ 0.          0.          0.          0.        ]
 [ 1.44269504  0.91023923  0.72134752  0.62133493]
 [ 4.32808512  2.73071768  2.16404256  1.8640048 ]
 [ 0.          0.          0.          0.        ]]
[[ 0.          1.44269504  4.32808512  0.        ]
 [ 0.          0.91023923  2.73071768  0.        ]
 [ 0.          0.72134752  2.16404256  0.        ]
 [ 0.          0.62133493  1.8640048   0.        ]]
[[ 0.          0.53245581  2.16404256  0.        ]
 [ 0.53245581  0.          0.37778341  0.28890429]
 [ 2.16404256  0.37778341  0.          0.30003776]
 [ 0.          0.28890429  0.30003776  0.        ]]


In [51]:
G = sigmoid_like * labels_diff * delta_dcg / max_dcg
print(G)

[[-0.         -0.         -0.         -0.        ]
 [-0.04575767 -0.         -0.         -0.02619837]
 [-0.20655866 -0.03965355 -0.         -0.03006946]
 [-0.         -0.         -0.         -0.        ]]


In [52]:
gradients = np.sum(G - G.T, axis=1)
print(gradients)

[ 0.25231633 -0.03230248 -0.27628167  0.05626783]


In [ ]:
class LambdaLoss(Function):
    
    @staticmethod
    def forward(ctx, input, target)